# Testing Misc Methods

## Dataloader with augmentation test

In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import imageio

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import torch.nn as nn
import glob

label_images_path = 'data/dataset/train/labels/*.png'
masks_images_path = 'data/dataset/train/masks'

filenames = sorted(glob.glob(label_images_path))

for filename in filenames:
    print(filename)
    img = imageio.imread(filename)
    img = img.astype(np.uint8)
    img = img * 255
    
    img = np.stack((img,img,img), axis=-1)
    print(img.shape)
    
    output_name = os.path.join(masks_images_path, os.path.basename(filename))
    imageio.imwrite(output_name, img)
    

In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import imageio

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import torch.nn as nn

from imgaug import parameters as iap
from imgaug import augmenters as iaa
import imgaug as ia


%matplotlib inline

# Custom includes
from dataloaders.alphapilot import AlphaPilotSegmentation
from dataloaders import utils

class Args():
    input_images_path = 'data/dataset/train/images'
    label_images_path = 'data/dataset/train/labels'

args = Args()
testBatchSize = 32
results_store_dir = 'data/test-aug'
imsize = 512

# augs_test = iaa.Sequential([
#     # Geometric Augs
#     iaa.Resize((imsize, imsize), interpolation='cubic'),
# #     iaa.Fliplr(0.5),
# #     iaa.Flipud(0.5),
# #     iaa.Rot90((0, 4)),
    
#     # Blur and Noise
# #     iaa.Sometimes(0.15, iaa.OneOf([iaa.GaussianBlur(sigma=(1.5, 2.5), name="gaus-blur"),
# #                                    iaa.MotionBlur(k=13, angle=[0, 180, 270, 360], direction=[-1.0, 1.0],
# #                                                  name='motion-blur'),
# #                                   ])),

#     # Color, Contrast, etc
# #     iaa.Sometimes(0.5, iaa.CoarseDropout(0.05, size_px=(2,4), per_channel=1.0, min_size=2, name='dropout')),
# #     iaa.SomeOf((0, None), [ iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 1.5), name="contrast")),    
# #                             iaa.Sometimes(0.5, iaa.Multiply((0.40, 1.60), per_channel=1.0, name="brightness")),
# #                             iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-30, 30), name="hue-sat")),
# #                           ]),
    
#     # Affine
#     iaa.Sometimes(0.99, iaa.Affine(scale={"x": (0.3, 0.7), "y": 1.0})),
#     iaa.Sometimes(0.99, iaa.Affine(scale=(0.3, 0.7))),
# ])

augs_test = iaa.Sequential([
        # Geometric Augs
        iaa.Resize((imsize, imsize), 0),
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Rot90((0, 4)),

        # Blur and Noise
        iaa.Sometimes(0.5, iaa.OneOf([iaa.GaussianBlur(sigma=(1.5, 2.5), name="gaus_blur"),
                                    iaa.MotionBlur(k=13, angle=[0, 180, 270, 360], direction=[-1.0, 1.0],
                                                    name='motion_blur'),
                                    ])),

        # Color, Contrast, etc
        iaa.Sometimes(0.90, iaa.CoarseDropout(0.05, size_px=(2, 4), per_channel=0.5, min_size=2, name='dropout')),

        iaa.SomeOf((0, None), [ iaa.Sometimes(0.9, iaa.GammaContrast((0.5, 1.5), name="contrast")),
                                iaa.Sometimes(0.9, iaa.Multiply((0.40, 1.60), per_channel=1.0, name="multiply")),
                                iaa.Sometimes(0.9, iaa.AddToHueAndSaturation((-30, 30), name="hue_sat")),
                            ]),

        # Affine
        iaa.Sometimes(0.5, iaa.Affine(scale={"x": (0.5, 0.7), "y": 1.0})),
        iaa.Sometimes(0.5, iaa.Affine(scale=(0.5, 0.7))),
        ])

db_test = AlphaPilotSegmentation(
    input_dir=args.input_images_path, label_dir=args.label_images_path,
    transform=augs_test,
    input_only=["gaus_blur", "motion_blur", "dropout", "contrast", "multiply",  "hue_sat"]
)
testloader = DataLoader(db_test, batch_size=testBatchSize, shuffle=False, num_workers=1, drop_last=True)

for ii, sample_batched in enumerate(testloader):
    inputs, labels, sample_filename = sample_batched
    print(sample_filename)
    
    labels_colormap = utils.decode_seg_map_sequence(labels.squeeze(1).numpy()).type(torch.FloatTensor)    
    
#     sample = torch.cat((inputs, labels_colormap), 0)
#     img_grid = make_grid(sample, nrow=4, padding=2)

    imgs_per_row = 8
    images = []
    for i in range (0, testBatchSize):
        images.append(inputs[i])
        images.append(labels_colormap[i])
    
    img_grid = make_grid(images, nrow=imgs_per_row, padding=2)
    
    save_image(img_grid, os.path.join(results_store_dir, sample_filename[0] + '-results.png'))


# Inference AlphaPilot Test

In [ ]:
import json
from pprint import pprint
import glob
import cv2
import numpy as np
from random import shuffle
import time
from matplotlib import pyplot as plt
%matplotlib inline

from inference_alphapilot import inferenceAlphaPilot


folder = 'data/dataset/test/images/'
img_file = glob.glob(folder + '*.JPG')
img_keys = [img_i.split('/')[-1] for img_i in img_file]

inference = inferenceAlphaPilot(checkpoint_path = 'checkpoint/checkpoint.pth')

time_all = []
pred_dict = {}
for img_key in img_keys:
    img =cv2.imread(folder+img_key)
    img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tic = time.monotonic()

    # Run inference
    mask = inference.inferenceOnNumpy(img)
    
#     # Resize Mask
    h, w = img.shape[:2]
    resized_mask = cv2.resize(mask, (w, h))
    print('resized:', resized_mask.shape)
    
    toc = time.monotonic()
    pred_dict[img_key] = resized_mask
    time_all.append(toc-tic)

mean_time = np.mean(time_all)
ci_time = 1.96*np.std(time_all)
freq = np.round(1/mean_time,2)
    
print('95% confidence interval for inference time is {0:.2f} +/- {1:.4f}.'.format(mean_time,ci_time))
print('Operating frequency from loading image to getting results is {0:.2f}.'.format(freq))






In [ ]:
import cv2
import numpy as np
import time

folder = 'data/dataset/test/images/'
img_file = glob.glob(folder + '*.JPG')
img_keys = [img_i.split('/')[-1] for img_i in img_file]

time_all = []
pred_dict = {}
tic1 = time.monotonic()
for img_key in img_keys:
    img =cv2.imread(folder+img_key)
    img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tic = time.monotonic()

    h, w = img.shape[:2]
    mask = cv2.resize(img, (512, 512))
    print(mask.shape)
    resized_mask = cv2.resize(mask, (w, h))
    print(resized_mask.shape)
    
    toc = time.monotonic()
    pred_dict[img_key] = resized_mask
    time_all.append(toc-tic)

mean_time = np.mean(time_all)
ci_time = 1.96*np.std(time_all)
freq = np.round(1/mean_time,2)

toc1 = time.monotonic()
print('Total time taken:', toc1-tic1)

print('95% confidence interval for inference time is {0:.2f} +/- {1:.4f}.'.format(mean_time,ci_time))
print('Operating frequency from loading image to getting results is {0:.2f}.'.format(freq))